In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import subprocess

In [4]:
# Set the path
data_dir = './MMHS150K/'
zipfile = 'MMHS150K.zip'
download_command = f"curl 'https://drive.usercontent.google.com/download?id=1S9mMhZFkntNnYdO-1dZXwF_8XIiFcmlF&export=download&authuser=0&confirm=t&uuid=db0e5b73-4ef4-45a4-b8f9-ef6f9c774473&at=APcmpozKaSM48fu1xNnp1-SNKDp1:1745766661322' > {zipfile}"

if not(os.path.isdir(data_dir) and os.listdir(data_dir)) and not(os.isfile(zipfile)):
    # Directory does not exists or is empty and zip file does not exist
    try:
        subprocess.run(['bash', '-c', download_command], check=True)
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error: {e}")
else:
    print("Data directory or zip file already exists.")

Data directory or zip file already exists.


In [ ]:
# Unzip file 
unzip_command = f"unzip {zipfile}"
if not(os.path.isdir(data_dir) and os.listdir(data_dir)) and os.isfile(zipfile):
    # Directory does not exists or is empty and zip file does not exist
    try:
        subprocess.run(['bash', '-c', unzip_command], check=True)
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error: {e}")
else:
    print("Data directory or zip file already exists.")

In [12]:
#load dataset
#json into dataframe
df = pd.read_json(os.path.join(data_dir, 'MMHS150K_GT.json'), lines=False, orient='index')

In [13]:
df.head()

,img_url,labels,tweet_url,tweet_text,labels_str
2005-04-28 09:09:13.714016256,http://pbs.twimg.com/tweet_video_thumb/D3gi9MH...,"[4, 1, 3]",https://twitter.com/user/status/11146793537140...,@FriskDontMiss Nigga https://t.co/cAsaLWEpue,"[Religion, Racist, Homophobe]"
2003-09-08 11:20:48.816660480,http://pbs.twimg.com/ext_tw_video_thumb/106301...,"[5, 5, 5]",https://twitter.com/user/status/10630200488166...,My horses are retarded https://t.co/HYhqc6d5WN,"[OtherHate, OtherHate, OtherHate]"
2005-02-20 19:22:48.075374593,http://pbs.twimg.com/media/D2OzhzHUwAADQjd.jpg,"[0, 0, 0]",https://twitter.com/user/status/11089273680753...,“NIGGA ON MA MOMMA YOUNGBOY BE SPITTING REAL S...,"[NotHate, NotHate, NotHate]"
2005-04-26 23:35:34.635618305,http://pbs.twimg.com/ext_tw_video_thumb/111401...,"[1, 0, 0]",https://twitter.com/user/status/11145585346356...,RT xxSuGVNGxx: I ran into this HOLY NIGGA TODA...,"[Racist, NotHate, NotHate]"
2002-10-22 02:08:00.215592966,http://pbs.twimg.com/media/Dl30pGIU8AAVGxO.jpg,"[1, 0, 1]",https://twitter.com/user/status/10352524802155...,“EVERYbody calling you Nigger now!” https://t....,"[Racist, NotHate, Racist]"


In [27]:
import json

with open("MMHS150K/MMHS150K_GT.json", "r") as f:
    raw_data = json.load(f)

with open("MMHS150K/mmhs150k_flat.jsonl", "w") as out:
    for tweet_id, entry in raw_data.items():
        entry["tweet_id"] = tweet_id
        out.write(json.dumps(entry) + "\n")

In [28]:
from datasets import load_dataset

full_ds = load_dataset("json", data_files="MMHS150K/mmhs150k_flat.jsonl",split="train" )

Generating train split: 0 examples [00:00, ? examples/s]

In [22]:
full_ds

Dataset({
    features: ['tweet_url', 'labels', 'img_url', 'tweet_text', 'labels_str', 'tweet_id'],
    num_rows: 149823
})

In [25]:
from datasets import DatasetDict
# Split dataset
def load_ids(path):
    with open(path, "r") as f:
        return set(line.strip() for line in f if line.strip())

train_ids = load_ids("MMHS150K/splits/train_ids.txt")
val_ids   = load_ids("MMHS150K/splits/val_ids.txt")
test_ids  = load_ids("MMHS150K/splits/test_ids.txt")

train_ds = full_ds.filter(lambda ex: ex["tweet_id"] in train_ids)
val_ds   = full_ds.filter(lambda ex: ex["tweet_id"] in val_ids)
test_ds  = full_ds.filter(lambda ex: ex["tweet_id"] in test_ids)

mmhs150k = DatasetDict({
    "train":      train_ds,
    "validation": val_ds,
    "test":       test_ds
})

In [29]:
mmhs150k

DatasetDict({
    train: Dataset({
        features: ['tweet_url', 'labels', 'img_url', 'tweet_text', 'labels_str', 'tweet_id'],
        num_rows: 134823
    })
    validation: Dataset({
        features: ['tweet_url', 'labels', 'img_url', 'tweet_text', 'labels_str', 'tweet_id'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['tweet_url', 'labels', 'img_url', 'tweet_text', 'labels_str', 'tweet_id'],
        num_rows: 10000
    })
})

In [30]:
mmhs150k.save_to_disk("MMHS150K/mmhs150k_HFDS")

Saving the dataset (0/1 shards):   0%|          | 0/134823 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]